In [23]:
# test SendGrid mailing list operations
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
params = {'page':1, 'page_size':1}
response = sg.client.contactdb.recipients.count.get()
print(response.status_code)
print(response.body)
#print(response.headers)

200
b'{"recipient_count":0}\n'


In [25]:
# Sendgrid add recipients
data = [
    {"email": "jstenger@uvm.edu"}
]
response = sg.client.contactdb.recipients.post(request_body=data)
print(response.status_code)
print(response.body)

201
b'{"new_count":1,"updated_count":0,"error_count":0,"error_indices":[],"unmodified_indices":[],"persisted_recipients":["anN0ZW5nZXJAdXZtLmVkdQ=="],"errors":[]}\n'


In [26]:
# Sendgrid retrieve recipients
params = {'page':1, 'page_size':1}
response = sg.client.contactdb.recipients.get(query_params=params)
print(response.status_code)
print(response.body)

200
b'{"recipient_count":1,"recipients":[{"id":"anN0ZW5nZXJAdXZtLmVkdQ==","email":"jstenger@uvm.edu","created_at":1753376141,"updated_at":1753376141,"last_emailed":null,"last_clicked":null,"last_opened":null,"first_name":null,"last_name":null}]}\n'


parse index.xml to get latest feed item

In [10]:
import feedparser

feed = feedparser.parse('../docs/index.xml')
newsletter = feed.entries[0]
print(feed.feed.title + ": " + newsletter.title)
print(newsletter.description)

newsletter: Post With Code
<p>This is a post with executable code.</p>


In [27]:
import sendgrid
import os
from sendgrid.helpers.mail import *
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
from_email = Email("brad.stenger@gmail.com")
to_email = To("jstenger@uvm.edu")
subject = feed.feed.title + ": " + newsletter.title
content = Content(MimeType.html, newsletter.description)
mail = Mail(from_email, to_email, subject, content)
response = sg.client.mail.send.post(request_body=mail.get())
print(response.status_code)
print(response.body)
print(response.headers)

202
b''
Server: nginx
Date: Thu, 24 Jul 2025 19:52:43 GMT
Content-Length: 0
Connection: close
X-Message-Id: RXScMViKTOWfq0fFvi1hgA
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin


